<a href="https://colab.research.google.com/github/aarabhim06/DynamicParkingPricing/blob/main/Aarabhi's_Final_Capstone_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Summer Analytics 2025 - Capstone Project**

## Problem: Dynamic Pricing for Urban Parking Lots  
We build a dynamic pricing engine for parking lots using real-time streaming data and behavioral demand features.

This notebook implements all three models:
- **Model 1: Baseline Linear Model**
- **Model 2: Demand-Based Model**
- **Model 3: Competitive Model**

**Libraries Used:** `Pathway`, `Bokeh`, `Pandas`, `NumPy`, `Panel`, `Scipy`

**Environment:** Google Colab


In [1]:
!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category20
from bokeh.layouts import layout
output_notebook()


## **Model 1 - Baseline Linear Pricing Model**

This is the first and simplest dynamic pricing strategy.  
It assumes that as the **occupancy increases**, the **price should linearly increase**.

### Formula:

$$
\text{Price} = \text{Base} + \alpha \cdot \frac{\text{Occupancy}}{\text{Capacity}}
$$

where:
- Base_Price = \$10
- α = 2 (scaling factor)
- Occupancy and Capacity are taken from the current streaming record

We integrate this logic into our **real-time data stream using Pathway**, which processes incoming parking records and computes the updated price live.



## Data Preprocessing

- Merged `Date` and `Time` into a `Timestamp`
- Selected one parking lot to simulate streaming for demo purposes


In [3]:
df = pd.read_csv("/content/dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Include lot ID so we can price per parking space
df[['ID', 'Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)


In [4]:
class ParkingSchema(pw.Schema):
    ID: str
    Timestamp: str
    Occupancy: int
    Capacity: int

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [5]:
data_with_time = data.with_columns(
    LotID = data.ID,
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [6]:
# Aggregation after window
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.LotID),  # Group by date and parking lot
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        total_occ = pw.reducers.sum(pw.this.Occupancy),
        total_cap = pw.reducers.sum(pw.this.Capacity),
        count = pw.reducers.count(),
        lot_id=pw.reducers.min(pw.this.ID)
    )
)

# First define average columns
delta_window = delta_window.with_columns(
    avg_occ = pw.this.total_occ / pw.this.count,
    avg_cap = pw.this.total_cap / pw.this.count,
)

# Then define price based on those averages
delta_window = delta_window.with_columns(
    price = 10 + 2 * (pw.this.avg_occ / pw.this.avg_cap)
)


In [9]:
model1_prices = delta_window.select(
    t=pw.this.t,
    ID=pw.this.lot_id,
    price_model1=pw.this.price
)


## **Final Plot - Model 1 Output**
This plot shows the dynamically calculated parking prices over time for all parking lots, based on daily occupancy patterns using our baseline pricing formula:

$$
\text{Price} = 10 + 2 \cdot \frac{\text{Average Occupancy}}{\text{Average Capacity}}
$$

where:
- X-axis: Time (daily timestamps across the 73-day period)

- Y-axis: Computed price for each parking lot (in dollars)

- Red circles: Each red dot represents the daily computed price for a specific parking lot at the end of that day’s time window

Currently, all parking lots are plotted together using the same color and marker style, so the graph may look visually crowded.



In [7]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Model 1: Real-Time Dynamic Pricing",
        x_axis_type="datetime"
    )
    fig.xaxis.axis_label = 'Time'
    fig.yaxis.axis_label = 'Price'
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [8]:
%%capture --no-display
pw.run()

Output()



> *This is Model 1 (Baseline) and serves as a reference. Upcoming models will incorporate more features like queue length, traffic, and competition.*




---



## **Model 2: Demand-Based Pricing Model**

In this model, we extend the baseline linear pricing by incorporating additional factors influencing parking demand. These include:

- Occupancy rate (Occupancy / Capacity)
- Queue length at the parking lot
- Nearby traffic condition
- Special day indicators (holidays, events)
- Vehicle type weights (different vehicle types impose varying demand)

The demand is computed using a weighted sum of these features, followed by normalization to ensure smooth and bounded price updates.

The price at each time step is then computed as:

$$
\text{Demand} = \alpha \cdot \frac{\text{Occupancy}}{\text{Capacity}} + \beta \cdot \text{QueueLength} - \gamma \cdot \text{Traffic} + \delta \cdot \text{IsSpecialDay} + \varepsilon \cdot \text{VehicleTypeWeight}
$$

where, the price is clipped to stay between 0.5x and 2x the BasePrice ($10).




In [10]:
df = pd.read_csv('/content/dataset.csv')  # Replace with your actual dataset path

# Convert Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Sort by Timestamp for streaming order
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map categorical features to numeric for Pathway processing:
# TrafficConditionNearby: low=0, average=1, high=2
traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['TrafficNumeric'] = df['TrafficConditionNearby'].map(traffic_map)

# VehicleType weights (example):
vehicle_weight_map = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5,
    'cycle': 0.3
}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weight_map)

# Save a CSV with all needed columns for streaming
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficNumeric', 'IsSpecialDay', 'VehicleWeight', 'ID']].to_csv(
    'streaming_data.csv', index=False)

### Calculating Demand and Pricing

We compute demand as a weighted sum of the features:

$$
\text{Demand} = \alpha \cdot \frac{\text{Occupancy}}{\text{Capacity}} + \beta \cdot \text{QueueLength} - \gamma \cdot \text{Traffic} + \delta \cdot \text{IsSpecialDay} + \varepsilon \cdot \text{VehicleTypeWeight}
$$


Constants (weights) are set as:

- α = 1.5
- β = 0.7
- γ = 0.5
- δ = 0.8
- ε = 1.0
- λ = 0.5 (price sensitivity)

After calculating demand, we normalize it and use it to adjust the price from a base value:

$$
\text{Price}_t = \text{BasePrice} \cdot \left(1 + \lambda \cdot \text{NormalizedDemand} \right)
$$

This ensures smooth price adjustments rather than abrupt spikes or drops.
- $\lambda$ is a scaling factor for demand influence.
- NormalizedDemand lies between 0 and 1.


In [12]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficNumeric: int
    IsSpecialDay: int
    VehicleWeight: float
    ID: str

data = pw.demo.replay_csv('streaming_data.csv', schema=ParkingSchema, input_rate=1000)

# Parse Timestamp to datetime and add day column
fmt = "%Y-%m-%d %H:%M:%S"

data = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

from datetime import timedelta

# Constants
alpha = 1.5
beta = 0.7
gamma = 0.5
delta = 0.8
epsilon = 1.0
lmbda = 0.5
base_price = 10

# Windowed aggregation of features
daily_agg = (
    data.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        avg_occ=pw.reducers.avg(pw.this.Occupancy),
        avg_cap=pw.reducers.avg(pw.this.Capacity),
        avg_queue=pw.reducers.avg(pw.this.QueueLength),
        avg_traffic=pw.reducers.avg(pw.this.TrafficNumeric),
        avg_special=pw.reducers.avg(pw.this.IsSpecialDay),
        avg_veh_weight=pw.reducers.avg(pw.this.VehicleWeight),
        lot_id=pw.reducers.min(pw.this.ID),
        window_end=pw.this._pw_window_end
    )
)

# Add raw demand calculation
with_demand = daily_agg.with_columns(
    occupancy_rate = pw.this.avg_occ / pw.this.avg_cap,
    raw_demand = (
        alpha * (pw.this.avg_occ / pw.this.avg_cap) +
        beta * pw.this.avg_queue -
        gamma * pw.this.avg_traffic +
        delta * pw.this.avg_special +
        epsilon * pw.this.avg_veh_weight
    )
)

# Use a bounded sigmoid-like function for price adjustment

final_model2 = with_demand.with_columns(
    unclipped_price = base_price * (
        1 + lmbda * (pw.this.raw_demand / (1 + pw.this.raw_demand))
    )
)

# Apply clipping
from typing import Any
# Clipping function
def clip_price(price: float) -> float:
    if price < 0.5 * base_price:
        return 0.5 * base_price
    elif price > 2 * base_price:
        return 2 * base_price
    return price

# Apply to Pathway column
final_model2 = final_model2.with_columns(
    price = pw.apply(clip_price, pw.this.unclipped_price)
)


---

## **Final Plot - Model 2 Output**

This plot illustrates the real-time dynamic pricing behavior for parking lots using a demand-sensitive pricing model.

- X-axis: Time (window_end) — end of each tumbling time window (e.g., 30 mins or 1 day)
- Y-axis: Computed parking price (in dollars) based on real-time demand
- Green line: Shows the evolution of price over time as demand fluctuates
- Orange scatter points: Represent individual price outputs emitted at each time window

The prices are clipped to stay within 0.5x and 2x the base price and are updated daily using a tumbling window approach.


In [13]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Model 2: Real-Time Demand-Based Pricing",
        x_axis_type="datetime"
    )
    fig.xaxis.axis_label = 'Time'
    fig.yaxis.axis_label = 'Price'
    fig.line("window_end", "price", source=source, line_width=2, color="green")
    fig.scatter("window_end", "price", source=source, size=5, color="orange") # Changed to scatter
    return fig

viz = final_model2.plot(price_plotter, sorting_col="window_end")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [14]:
%%capture --no-display
pw.run()


Output()

In [15]:
model2_prices = final_model2.select(
    t=pw.this.window_end,  # Renamed window_end to t for consistency
    ID=pw.this.lot_id,
    price_model2=pw.this.price
)

> ***Model 2 improves on Model 1*** *by reacting to real-time features like traffic, queue length, special events, and vehicle type. This makes the pricing more context-aware and efficient.*



---



###**Joining Models for Comparison**
Since both models share the same LotID and timestamp (t), we join them using Pathway to enable side-by-side comparison.

Using Bokeh, we create an interactive plot comparing the two pricing strategies:
- Model 1: Navy line with red markers
- Model 2: Green line with orange markers

This allows us to visually inspect the behavior of both models across time, identifying patterns, volatility, and alignment.

In [23]:
import panel as pn
import bokeh.plotting

# Join the two tables
merged = model1_prices.join(
    model2_prices,
    pw.left.t == pw.right.t,
    pw.left.ID == pw.right.ID
)

# Select needed columns from the joined table, referencing them directly
merged = merged.select(
    t=pw.right.t, # Access 't' from the (right =) left table
    price_model1=pw.left.price_model1, # Access price from model1
    price_model2=pw.right.price_model2 # Access price from model2
)

# Define combined plotter
def combined_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Baseline Linear (Model 1) vs Demand-Based (Model 2): Price Comparison",
        x_axis_type="datetime"
    )
    fig.xaxis.axis_label = 'Time'
    fig.yaxis.axis_label = 'Price'

    # Model 1 line
    fig.line("t", "price_model1", source=source, line_width=2, color="navy", legend_label="Baseline Linear")
    fig.scatter("t", "price_model1", source=source, size=5, color="red") # Changed to scatter

    # Model 2 line
    fig.line("t", "price_model2", source=source, line_width=2, color="green", legend_label="Demand-Based")
    fig.scatter("t", "price_model2", source=source, size=5, color="orange") # Changed to scatter

    fig.legend.location = "center_left"
    return fig

# Plot using merged Pathway table
viz = merged.plot(combined_plotter, sorting_col="t")
pn.Column(viz).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [24]:
%%capture --no-display
pw.run()

Output()

---

### **Model 3: Competitive Pricing with Location and Competition Awareness**

In this model, we enhance the pricing engine by incorporating spatial proximity and competitor prices.


- Calculate distances between parking lots using their latitude and longitude.
- Consider competitor prices in nearby lots to adjust our own prices.
- If our lot is **full** and nearby lots are cheaper, **suggest rerouting** or reduce price.
- If competitors are expensive, we can increase price while remaining attractive.
- This simulates real-world market competition and optimizes utilization.

We'll implement this logic using Pathway's streaming capabilities and integrate it with the pricing calculations from Model 2.


In [56]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time into a single Timestamp column
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Drop duplicates or sort if needed
df = df.sort_values("Timestamp").reset_index(drop=True)

# Convert categorical traffic conditions to numeric scale
traffic_map = {"low": 1, "average": 2, "high": 3}
df["TrafficNumeric"] = df["TrafficConditionNearby"].map(traffic_map)

# Encode vehicle type as weight
vehicle_weights = {
    "cycle": 0.3,
    "bike": 0.5,
    "car": 1.0,
    "truck": 1.5
}
df["VehicleWeight"] = df["VehicleType"].map(vehicle_weights)

# Save required columns to a streaming-friendly CSV
df_stream = df[[
    "Timestamp",
    "Occupancy",
    "Capacity",
    "QueueLength",
    "TrafficNumeric",
    "IsSpecialDay",
    "VehicleWeight",
    "ID",
    "Latitude",
    "Longitude"
]]

df_stream.to_csv("streaming_data2.csv", index=False)


In [57]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficNumeric: int
    IsSpecialDay: int
    VehicleWeight: float
    ID: str
    Latitude: float
    Longitude: float

data = pw.demo.replay_csv("streaming_data2.csv", schema=ParkingSchema, input_rate=1000)
locations_df = df_stream[["ID", "Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)

We use the **Haversine formula** to compute the great-circle distance between two geographic points (latitude and longitude).

This allows us to:

- Find nearby competitor lots within a certain radius (e.g., 1 km).
- Use these distances to incorporate competitor prices into our own pricing logic.


In [28]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in km
    R = 6371.0

    # Convert coordinates to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c  # Distance in kilometers


In [29]:
# Find competitors within 0.5 km
from collections import defaultdict

radius_km = 0.5
competitors = defaultdict(list)

from scipy.spatial import KDTree
import numpy as np
from collections import defaultdict

# Extract coordinates and IDs
coordinates = locations_df[["Latitude", "Longitude"]].values
ids = locations_df["ID"].values

# Build KDTree
tree = KDTree(np.radians(coordinates))  # KDTree needs radians if using haversine

# Define radius in radians (for Earth)
EARTH_RADIUS_KM = 6371
radius_km = 0.5
radius_rad = radius_km / EARTH_RADIUS_KM

# Map lot IDs to competitor IDs
competitors = defaultdict(list)

for i, (lat, lon) in enumerate(np.radians(coordinates)):
    idxs = tree.query_ball_point([lat, lon], r=radius_rad)
    for j in idxs:
        if i != j:
            competitors[ids[i]].append(ids[j])


In [30]:
import random

avg_comp_prices = {
    lot_id: round(random.uniform(7, 13), 2)
    for lot_id in locations_df["ID"]
}

def get_avg_comp_price(lot_id):
    comp_ids = competitors.get(lot_id, [])
    comp_prices = [avg_comp_prices.get(cid, 10) for cid in comp_ids]
    if comp_prices:
        return np.mean(comp_prices)
    else:
        return 10.0  # Default competitor price if none nearby


In [31]:
base_price = 10

def competitive_price_formula(
    occupancy, capacity, queue_len, traffic, special_day, comp_price
):
    occ_rate = occupancy / capacity if capacity > 0 else 0.5

    # Demand score
    demand_score = (
        2.0 * occ_rate +
        0.4 * queue_len -
        0.3 * traffic +
        0.6 * special_day
    )

    # Price based on demand
    price = base_price + 2 * demand_score

    # Adjust based on competition
    if price > comp_price + 1:
        price = price - 1.0  # You're overpriced
    elif price < comp_price - 1:
        price = price + 0.8  # You're underpriced --> raise slightly

    # Clip to bounds
    price = max(0.5 * base_price, min(2 * base_price, price))
    return round(price, 2)


In [32]:
final_model3 = data.windowby(
    pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    instance=pw.this.ID,
    window=pw.temporal.tumbling(timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    ID = pw.reducers.min(pw.this.ID),
    avg_occ = pw.reducers.avg(pw.this.Occupancy),
    avg_cap = pw.reducers.avg(pw.this.Capacity),
    avg_queue = pw.reducers.avg(pw.this.QueueLength),
    avg_traffic = pw.reducers.avg(pw.this.TrafficNumeric),
    avg_special = pw.reducers.avg(pw.this.IsSpecialDay),
    window_end = pw.this._pw_window_end
).with_columns(
    price = pw.apply(
        competitive_price_formula,
        pw.this.avg_occ,
        pw.this.avg_cap,
        pw.this.avg_queue,
        pw.this.avg_traffic,
        pw.this.avg_special,
        pw.apply(get_avg_comp_price, pw.this.ID)
    )
)


### Competitive Pricing Explanation

In this model, we incorporate real-time competitor analysis by computing the distance between lots and observing the price trends of nearby locations. If nearby lots are full or charging more, our lot adjusts prices upward. Conversely, if nearby lots are cheaper, our lot reduces price or suggests rerouting (not implemented yet, optional).

This makes the pricing behavior **market-aware** and aligns better with how pricing works in urban settings with competing parking zones.


In [33]:
model3_prices = final_model3.select(
    t = pw.this.window_end,
    ID = pw.this.ID,
    price_model3 = pw.this.price
)


In [48]:
def model3_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Model 3: Competitive Geo-Based Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price_model3", source=source, line_width=2, color="purple")
    fig.circle("t", "price_model3", source=source, size=3.5, color="violet")
    return fig

viz = model3_prices.plot(model3_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [49]:
%%capture --no-display
pw.run()

Output()

In [51]:
joined_12 = model1_prices.join(
    model2_prices,
    pw.left.t == pw.right.t,
    pw.left.ID == pw.right.ID
)
joined_12 = joined_12.select(
    t = pw.left.t,
    ID = pw.left.ID,
    price_model1 = pw.left.price_model1,
    price_model2 = pw.right.price_model2
)
merged_all = joined_12.join(
    model3_prices,
    pw.left.t == pw.right.t,
    pw.left.ID == pw.right.ID
)
merged_all = merged_all.select(
    t = pw.left.t,
    ID = pw.left.ID,
    price_model1 = pw.left.price_model1,
    price_model2 = pw.left.price_model2,
    price_model3 = pw.right.price_model3
)


In [54]:
def combined_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="All Models: Dynamic Pricing Comparison",
        x_axis_type="datetime"
    )
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price"

    fig.line("t", "price_model1", source=source, line_width=2, color="navy", legend_label="Model 1")
    fig.line("t", "price_model2", source=source, line_width=2, color="green", legend_label="Model 2")
    fig.line("t", "price_model3", source=source, line_width=2, color="red", legend_label="Model 3")

    fig.legend.location = "top_left"
    return fig

viz_all = merged_all.plot(combined_plotter, sorting_col="t")
pn.Column(viz_all).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [55]:
%%capture --no-display
pw.run()


Output()



## **Explanation of Pricing Behavior Across Models**

### **Model 1 (Base Linear Pricing):**
* This is the simplest pricing model. It adjusts prices primarily based on the average occupancy ratio of the parking lot.

* Because it uses a straightforward linear relationship and does not account for detailed demand factors or competitor behavior, its prices tend to be more stable and conservative;often the lowest among the three models.

* It provides a baseline price that reflects basic utilization without reacting aggressively to demand fluctuations.


### **Model 2 (Demand-Based Pricing):**
* This model incorporates multiple real-world demand signals such as queue length, traffic conditions, special days, and vehicle type weights.

* The inclusion of these factors allows Model 2 to dynamically adjust prices more responsively and aggressively according to demand intensity.

* Additionally, a clipping function ensures prices stay within a reasonable range but allows peaks to be higher than Model 1's.

* As a result, Model 2 typically produces higher prices reflecting scarcity and demand surges.


### **Model 3 (Competitive Pricing with Location Intelligence):**  

* Building on Model 2's demand-driven prices, Model 3 introduces competitor-aware adjustments based on geographic proximity and competitor prices.

* It nudges prices slightly down if nearby lots are cheaper (to stay competitive) or up if competitors are more expensive (to capitalize on demand).

* This causes Model 3 prices to fluctuate around Model 2's values, generally positioning it between Model 1 and Model 2.

* This model simulates realistic market competition and dynamic pricing strategies observed in actual parking ecosystems.

---

### **Summary:**
Model 1 sets a stable baseline, Model 2 reflects detailed demand-driven price optimization, and Model 3 introduces market-aware competitive adjustments, resulting in a nuanced pricing spectrum with Model 3 often lying between the conservative Model 1 and the demand-sensitive Model 2.


